In [9]:
import pandas as pd
import requests
from string import Template
import time
import hmac

In [2]:
VERSION = '1.1'
BASE_REQUEST = Template('https://bittrex.com/api/v$version/$scope/')

PUBLIC_BASE_URL = BASE_REQUEST.substitute(version=VERSION, scope='public')
MARKET_BASE_URL = BASE_REQUEST.substitute(version=VERSION, scope='market')
ACCOUNT_BASE_URL = BASE_REQUEST.substitute(version=VERSION, scope='account')

API_KEY = ''
SECRET_KEY = ''

PUBLIC_METHODS = ['getmarkets', 'getcurrencies', 'getticker', 'getmarketsummaries', 'getmarketsummary', 'getorderbook',
                  'getmarkethistory']

MARKET_METHODS = ['buylimit', 'selllimit', 'cancel', 'getopenorders']

ACCOUNT_METHODS = ['getbalances', 'getbalance', 'getdepositaddress', 'withdraw', 'getorder', 'getorderhistory', 
                   'getwithdrawalhistory', 'getdeposithistory']

In [33]:
# Method for unpacking arguments and forming them into a formatted string that can be used in a Bittrex API request 
def unpack_args(args):
    formatted_args = ''
    
    if args != None:
        for k in args.keys():
            formatted_args += (k + '=' + args[k]+ '&')
            
    return formatted_args


# Wrapper method for accessing the Bittrex API
def bittrex_request(scope, method, args=None):

    if scope == PUBLIC_BASE_URL:
        request_url_template = Template('$scope$method?$args')
        request_url = request_url_template.substitute(scope=scope, method=method, args=unpack_args(args))
        return pd.DataFrame(requests.get(request_url).json()['result'])
    
    elif scope == 'market':
        request_url_template = Template('$scope$method?$args')
        nonce = time.time()
        request_url = request_url_template.substitute(scope=scope, method=method, args=unpack_args(args))
    
    elif scope == 'account':
        request_url_template = Template('$scope$method?$args')
        nonce = time.time()
        request_url = request_url_template.substitute(scope=scope, method=method, args=unpack_args(args))
    
    

In [35]:
# args = {'type': 'sell', 'market': 'BTC-XRP'}
args = {'market': 'BTC-ETH'}
bittrex_request(PUBLIC_BASE_URL, 'getmarketsummaries').sort_values(by=['OpenBuyOrders'], ascending=False)

,Ask,BaseVolume,Bid,Created,High,Last,Low,MarketName,OpenBuyOrders,OpenSellOrders,PrevDay,TimeStamp,Volume
250,4.535000e+03,4.793951e+07,4.535000e+03,2015-12-11T06:31:40.633,4.566000e+03,4.535000e+03,4.110000e+03,USDT-BTC,4693,1723,4.162416e+03,2017-09-06T10:57:26.213,1.097304e+04
110,4.935000e-03,1.021273e+04,4.927120e-03,2016-10-26T01:28:31.96,5.600000e-03,4.927120e-03,4.700000e-03,BTC-NEO,4415,23481,5.201410e-03,2017-09-06T10:57:26.433,1.963837e+06
192,1.530000e-06,3.663330e+03,1.520000e-06,2016-02-17T22:42:03.61,1.780000e-06,1.530000e-06,1.420000e-06,BTC-XVG,4230,10510,1.610000e-06,2017-09-06T10:57:25.84,2.297594e+09
118,2.418900e-03,9.502086e+03,2.411420e-03,2017-07-16T17:07:42.313,2.634990e-03,2.411420e-03,2.166610e-03,BTC-OMG,4034,4729,2.247980e-03,2017-09-06T10:57:28.887,3.914093e+06
62,7.300000e-02,5.889980e+03,7.297000e-02,2015-08-14T09:02:24.817,7.450000e-02,7.300000e-02,7.000051e-02,BTC-ETH,3339,12370,7.044900e-02,2017-09-06T10:57:28.043,8.176987e+04
15,1.287890e-01,1.450967e+03,1.280351e-01,2017-08-01T18:34:04.967,1.298800e-01,1.289890e-01,1.221473e-01,BTC-BCC,2466,14261,1.228519e-01,2017-09-06T10:57:27.963,1.160787e+04
97,1.716204e-02,3.068907e+03,1.711000e-02,2014-02-13T00:00:00,1.727499e-02,1.711000e-02,1.596900e-02,BTC-LTC,2455,5595,1.596808e-02,2017-09-06T10:56:47.333,1.841995e+05
189,5.065000e-05,1.703288e+03,5.056000e-05,2014-12-22T19:30:27.45,5.067000e-05,5.056000e-05,4.850000e-05,BTC-XRP,2250,19957,4.922000e-05,2017-09-06T10:57:28.293,3.438425e+07
135,3.200000e-07,5.003292e+02,3.100000e-07,2014-02-25T09:00:00,3.400000e-07,3.100000e-07,2.700000e-07,BTC-RDD,2153,13870,2.900000e-07,2017-09-06T10:57:11.727,1.626298e+09
187,2.630981e-02,7.415246e+02,2.614073e-02,2014-06-04T07:38:39.24,2.755997e-02,2.630980e-02,2.474111e-02,BTC-XMR,2140,4187,2.510011e-02,2017-09-06T10:57:17.933,2.782542e+04


# Auth up!

In [10]:
api_key = open('keys.txt', 'r').read().split('\n')[0].split(':')[1].strip()
secret_key = open('keys.txt', 'r').read().split('\n')[1].split(':')[1].strip()

nonce = time.time()

request_url = 'https://bittrex.com/api/v1.1/account/getbalances?apikey=' + api_key + '&nonce=' + str(nonce);

# Sign the request
sign = hmac.new(secret_key, request_url, hashlib.sha512).hexdigest()

# Add the sign to the request header
request_headers = {'apisign': sign}

# Make the request
authd_response = requests.get(request_url, headers = request_headers)

authd_response.content

IOError: [Errno 2] No such file or directory: 'keys.txt'